Nesta aula, vamos ver um exemplo de problema de classificação. Vamos utilizar um conjunto de dados sobre qualidade da água. Mais informações sobre esse conjunto de dados podem ser obtidas [aqui](https://www.kaggle.com/datasets/adityakadiwal/water-potability).

Preparando o ambiente

In [ ]:
# Clonando pasta do github
if 'google.colab' in str(get_ipython()):
    !git clone https://github.com/tiagofiorini/MLinPhysics.git
    import os as os
    os.chdir('./MLinPhysics')

In [2]:
# Carregando bibliotecas básicas
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
# Leitura do arquivo de dados
df = pd.read_csv('aula12_dados_water_potability.csv', header = 0, decimal = '.', sep = ",")

# Análise Exploratória

Observe que todas as variáveis preditoras (features) são numéricas. Há valores faltantes em algumas delas. A variável alvo está codificada em 0 e 1.

In [ ]:
df.info()

In [ ]:
df.head()

Há dados faltantes?

In [ ]:
# Há dados faltantes?
# Counting NaN values in all columns
print(df.isna().sum())
# Heatmap com a distribuição de valores faltantes
sns.heatmap(df.isnull())

In [ ]:
# Se utilizarmos apenas as linhas com dados completos, quantas linhas vão sobrar?
df_clean = df.dropna(axis = 0, how = 'any', inplace = False)
print(len(df_clean), 'linhas restantes de um total de', len(df), 'linhas')
print(100*len(df_clean)/len(df),'%')

*Exercício: testar quantas linhas completas sobram se exluirmos a coluna Sulfate, que tem a maioria dos dados faltantes.*

**Como é a distribuição da variável alvo?**

Veja que há predominância da classe não-potável, mas não há um desbalanço importante entre as classes.

In [ ]:
print('Potability=1: ', len(df[df.Potability==1]), ',' , 100*len(df[df.Potability==1])/len(df), '%')
print('Potability=0: ', len(df[df.Potability==0]), ',' , 100*len(df[df.Potability==0])/len(df), '%')

**Variáveis numéricas**

Observe que as features possuem vários outliers, que poderiam ser "tratados" na etapa de preparação de dados.

Observe também que não há grande diferença na distribuição das variáveis considerando amostras potáveis e não potáveis.

Observe também que as variáveis numéricas não são correlacionadas.

In [ ]:
# Distribuição de valores em amostras potáveis e não-potáveis
# Boxplots para uma feature
sns.boxplot(y='ph', x='Potability', data=df, notch=True)

In [ ]:
# Boxplots para todas as features
names = df.drop('Potability', axis=1).columns
fig, axes = plt.subplots(3,3)
for name, ax in zip(names, axes.flatten()):
    sns.boxplot(y=name, x='Potability', data=df, notch=True, ax=ax)
plt.tight_layout()

In [ ]:
# Avaliando correlações
sns.pairplot(df.drop('Potability',axis=1))

In [ ]:
sns.heatmap(df.drop('Potability',axis=1).corr(), annot=True, cmap='BrBG')

# Preparação dos dados

*Exercício: avalie mudanças na estrutura e no desempenho da árvore para diferentes procedimentos de preparação de dados.*
*   *Utilizar outra estratégia para lidar com os dados faltantes*
*   *Amostragem estratificada, mantendo a proporção de amostras potáveis e não-potáveis nos conjuntos de treino e teste*







In [10]:
# Separação de variáveis preditoras e alvo
X = df_clean.drop(['Potability'], axis=1) # features
y = df_clean.Potability # target

In [11]:
# Particionamento em treino e teste
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2) #, random_state=18)

# Árvore de decisão sem poda

*Exercício: experimente outros critérios de divisão: “gini”, “entropy”, “log_loss”. Avalie se muda a estrutura e o desempenho da árvore.*

In [16]:
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree

# Create classifier
clf = DecisionTreeClassifier(criterion='gini',
                             splitter='best',
                             random_state=6)
model = clf.fit(X_train, y_train)

# Imprimindo a árvore como texto
# print(tree.export_text(clf, feature_names=list(X.columns)))

In [ ]:
# Imprimindo a árvore como um fluxograma
width = 10
height = 7
plt.figure(figsize=(width, height))

tree.plot_tree(clf, feature_names=list(X.columns), class_names=['N','Y'], filled=True)
plt.show()

Desempenho do modelo. Observe os valores de diferentes métricas: acurácia, precisão, recall, F1 score.

In [ ]:
# Matriz de confusão para o conjunto de teste
# Output: array
from sklearn.metrics import confusion_matrix, classification_report

y_pred = clf.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
print('Matriz de confusão:')
print(cm)
print('Relatório:')
print(classification_report(y_test, y_pred, target_names=['No','Yes']))

In [ ]:
# Matriz de confusão para o conjunto de validação
# Output: heatmap
from sklearn.metrics import ConfusionMatrixDisplay
y_pred = clf.predict(X_test)
ConfusionMatrixDisplay.from_predictions(y_test, y_pred,
                                        display_labels=['Não potável','Potável'],
                                        cmap=plt.cm.Blues)

# Árvore de decisão com restrições

Veja [aqui](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html) como configurar os hiperparâmetros.

*Exercício: variar os seguintes hiperparâmetros, avaliando mudanças na estrutura e no desempenho da árvore*

*   *max_depth: maximum depth of the tree*
*   *max_features: number of features to consider for the best split*
*   *min_samples_split: minimum number of samples required to split an internal node*
*   *min_samples_leaf: minimum number of samples in a leaf node*







In [ ]:
# Create classifier
clf = DecisionTreeClassifier(criterion='gini',
                             splitter='best',
                             max_depth=None, # default
                             max_features=None,
                             min_samples_split=2,
                             min_samples_leaf=1)
model = clf.fit(X_train, y_train)

# Imprimindo a árvore como texto
print(tree.export_text(clf, feature_names=list(X.columns)))

In [ ]:
# Imprimindo a árvore como um fluxograma
width = 10
height = 7
plt.figure(figsize=(width, height))

tree.plot_tree(clf, feature_names=list(X.columns), class_names=['N','Y'], filled=True, impurity=False)
plt.show()

In [ ]:
# Desempenho no conjunto de teste
from sklearn.metrics import classification_report, ConfusionMatrixDisplay

y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred, target_names=['Não potável','Potável']))
ConfusionMatrixDisplay.from_predictions(y_test, y_pred,
                                        display_labels=['Não potável','Potável'],
                                        cmap=plt.cm.Blues)

# Árvore de decisão: otimização de hiperparâmetros

In [ ]:
# Otimização usando GridSearch
from sklearn.model_selection import GridSearchCV

clf = DecisionTreeClassifier(criterion='gini', splitter='best')

params = [{'max_depth': [5, 8, 11],
         'max_features': [3, 6, 9],
         'min_samples_split': [0.05, 0.1, 0.2], # 0.05*n_samples
         'min_samples_leaf': [0.025, 0.05, 0.1]}]

gs_tree = GridSearchCV(estimator = clf,
                      param_grid = params,
                      scoring = 'accuracy') # 'recall', 'precision', 'f1', 'accuracy'
gs_tree.fit(X_train, y_train)

# Imprimindo os resultados.
# pd.DataFrame(gs_tree.cv_results_)
print('Melhores hiperparâmetros:')
print(gs_tree.best_params_)
print('Melhor score:')
print(gs_tree.best_score_)

*Escreva os valores dos melhores hiperparâmetros no código abaixo.*

In [ ]:
# Treinando o modelo com os hiperparâmetros otimizados
clf = DecisionTreeClassifier(criterion='gini',
                             splitter='best',
                             max_depth=None,
                             max_features=None,
                             min_samples_split=2,
                             min_samples_leaf=1)
model = clf.fit(X_train, y_train)

# Imprimindo a árvore como texto
# print(tree.export_text(clf, feature_names=list(X.columns)))
# Imprimindo a árvore como um fluxograma
width = 20
height = 14
plt.figure(figsize=(width, height))
tree.plot_tree(clf, feature_names=list(X.columns), class_names=['N','Y'], filled=True, impurity=False)
plt.show()

In [ ]:
# Desempenho no conjunto de teste
from sklearn.metrics import classification_report, ConfusionMatrixDisplay

y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred, target_names=['Não potável','Potável']))
ConfusionMatrixDisplay.from_predictions(y_test, y_pred,
                                        display_labels=['Não potável','Potável'],
                                        cmap=plt.cm.Blues)

# Random Forest

Treinando o classificador Randon Forest com valores padrão para os hiperparâmetros. Veja [aqui](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) o significado dos hiperparâmetros.

In [ ]:
from sklearn.ensemble import RandomForestClassifier
clf_rf = RandomForestClassifier(n_estimators=100, criterion='gini', bootstrap=True,
                             max_depth=None, # default
                             min_samples_split=2,
                             min_samples_leaf=1,
                             min_weight_fraction_leaf=0.0,
                             max_features='sqrt',
                             max_leaf_nodes=None,
                             min_impurity_decrease=0.0)
clf_rf.fit(X_train, y_train)

In [ ]:
# Desempenho no conjunto de teste
from sklearn.metrics import classification_report, ConfusionMatrixDisplay

y_pred = clf_rf.predict(X_test)

print(classification_report(y_test, y_pred, target_names=['Não potável','Potável']))
ConfusionMatrixDisplay.from_predictions(y_test, y_pred,
                                        display_labels=['Não potável','Potável'],
                                        cmap=plt.cm.Blues)

*Exercício: otimizar uma seleção de hiperparâmetros utilizando o método GridSearch. Treinar o classificador Random Forest com os hiperparâmetros otimizados. Verificar o desempenho do modelo.*